### 车辆统计项目

### 1.加载视频

In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture('./video.mp4')

# 循环读取视频帧
while True:
    ret,frame=cap.read()
    if ret ==True:
        cv2.imshow('video',frame)
        
    key = cv2.waitKey(1)
    # 用户按esc退出
    if key ==27:
        break

cap.release()
cv2.destroyAllWindows()

### 2.去除背景示例

In [3]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
# 创建mog对象
mog = cv2.bgsegm.createBackgroundSubtractorMOG()

while True:
    ret, frame = cap.read()
    if ret == True:
        fgmask = mog.apply(frame)
        cv2.imshow('video',fgmask)
        
    key = cv2.waitKey(1)
    # 用户按esc退出
    if key ==27:
        break

cap.release()
cv2.destroyAllWindows()

### 3.形态学识别车辆

In [29]:
import cv2
import numpy as np

cap = cv2.VideoCapture('./video.mp4')

# 创建mog对象
mog = cv2.bgsegm.createBackgroundSubtractorMOG()
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))

min_w = 80
min_h = 80

line_high = 620
# 偏移量
offset = 9

cars = []
carno =0

# 计算外接矩形的中心点
def center(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = int(x)+x1
    cy = int(y)+y1
    return cx,cy

# 循环读取视频帧
while True:
    ret,frame=cap.read()
    if ret ==True:
        # 把原始帧进行灰度化，然后去噪
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        # 去噪
        blur = cv2.GaussianBlur(gray,(3,3),5)
        mask = mog.apply(blur)
        
        # 腐蚀
        erode = cv2.erode(mask,kernel)
#         cv2.imshow('video',mask)
        # 膨胀，把图像还原回来
        dialte = cv2.dilate(erode,kernel,iterations=2)
        
        # 消除内部的小块
        # 闭运算
        close = cv2.morphologyEx(dialte,cv2.MORPH_CLOSE,kernel)
        
        # 查找轮廓
        contours, h=cv2.findContours(close,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        
        cv2.line(frame,(10,line_high),(1200,line_high),(255,255,0),3)
        
        #画出所有检测出来的轮廓
        for contour in contours:
            # 最大外接矩形
            (x,y,w,h) = cv2.boundingRect(contour)
            #通过外接矩形的宽高大小来过滤
            is_valid = (w>=min_w)and(h>=min_h)
            if not is_valid:
                continue
            
            # 能走到这里来的都是符合要求的矩形，即正常的车
            # 要求坐标都是整数
            cv2.rectangle(frame,(int(x),int(y)),(int(x+w),int(y+h)),(0,0,255),2)
            # 把车抽象为一点，即外接矩形的中心点
            # 要通过外接矩形计算矩形的中心点
            cpoint = center(x,y,w,h)
            cars.append(cpoint)
            cv2.circle(frame,(cpoint),5,(0,0,255),-1)
            
            #判断汽车是否过线
            for(x,y) in cars:
                if y>(line_high-offset) and y < (line_high +offset):
                    # 落入了有效区间
                    # 计数加1
                    carno+=1
                    cars.remove((x,y))
#                     print(carno)
        # 如何划线,画在哪
        # 如何去计数,
#         cv2.imshow('video',erode)
#         cv2.imshow('dialte',dialte)
#         cv2.imshow('close',close)
        cv2.putText(frame,'Vehicle Count:'+str(carno),(500,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),5)
        cv2.imshow('frame',frame)
    
    key = cv2.waitKey(30)
    # 用户按esc退出
    if key ==27:
        break

# 出现一个车计算两个点的情况,原因应该是汽车在行驶的过程中,中点发生变化
cap.release()
cv2.destroyAllWindows()